<a href="https://colab.research.google.com/github/shriram9992/anomaly_detection/blob/master/shuttle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving shuttle.csv to shuttle (1).csv


In [0]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['shuttle.csv']),header=None)

In [3]:
nor1 = df2.loc[df2[9]==1]
nor2 = df2.loc[df2[9]==4]
nor = nor1.append(nor2)
nor.reset_index(inplace = True,drop = True)
print(nor.head())

    0  1    2  3   4   5   6   7  8  9
0  53  0   82  0  52  -5  29  30  2  1
1  37  0   76  0  28  18  40  48  8  1
2  37  0   79  0  34 -26  43  46  2  1
3  55 -1   95 -3  54  -4  40  41  2  1
4  37  0  101 -7  28   0  64  73  8  1


In [4]:
ano1 = df2.loc[df2[9]==2]
ano2 = df2.loc[df2[9]==3]
ano3 = df2.loc[df2[9]==5]
ano = ano1.append(ano2)
ano = ano.append(ano3)
ano.reset_index(inplace = True,drop = True)
print(ano.head())

    0   1   2  3   4  5   6    7    8  9
0  50  21  77  0  28  0  27   48   22  2
1  70  57  82  0 -42 -4  12  126  114  2
2  42  36  77  0 -22  0  36  101   64  2
3  69  59  82  0 -42 -5  13  126  112  2
4  68  67  82  0 -42 -6  13  126  112  2


In [5]:
import numpy as np
nor.apply(np.random.shuffle,axis=0)
print(nor.head())

    0  1   2  3   4   5   6   7   8  9
0  37  0  77  0  30  -2  67  35  10  1
1  37  0  81  0  52  16  40  49   0  4
2  56 -2  77  0  44 -11  35  55   2  1
3  46  0  81  0  54  -3  67  41   0  4
4  37  0  77  0   2   0  47  35   0  1


In [6]:
ano.apply(np.random.shuffle,axis=0)
print(ano.head())

     0  1    2  3   4   5   6    7   8  9
0  104  2  108 -2  -4   0   2  125  34  5
1  106  2   88  1  -2  29   3  114  86  5
2  101 -1  105  0  62   0   1   81  64  5
3  105 -5   83  3 -42  15  30   33  34  5
4  105 -1   84 -4  70   0   1   92  32  5


In [7]:
X_train = nor.loc[0:43592,:]
X_train = X_train.drop(9,1)
X_train.reset_index(inplace = True,drop = True)
print(X_train.head())

    0  1   2  3   4   5   6   7   8
0  37  0  77  0  30  -2  67  35  10
1  37  0  81  0  52  16  40  49   0
2  56 -2  77  0  44 -11  35  55   2
3  46  0  81  0  54  -3  67  41   0
4  37  0  77  0   2   0  47  35   0


In [8]:
X_test = nor.loc[43592:54489,:]
X_test1 = ano.loc[:,:]
X_test = X_test.append(X_test1)
X_test.reset_index(inplace = True,drop = True)
X_test = X_test.drop(9,1)
print(X_test.head())

    0  1    2  3   4   5   6   7   8
0  55  0  110  0  50 -12  45  37  50
1  45  0   86  0  42 -16  43  79   0
2  43  0   89  0  42   0  45  47   0
3  43  0   80 -3  52  -6  41  58   0
4  37 -1   79  0  46  30  53  38   2


In [9]:
y_test1 = nor.loc[43592:54489,9]
y_test2 = ano.loc[:,9]
y_test = y_test1.append(y_test2)
y_test.reset_index(inplace = True,drop = True)
y_test = y_test.map({1:1,4:1,2:-1,3:-1,5:-1})
print(y_test.head())


0    1
1    1
2    1
3    1
4    1
Name: 9, dtype: int64


In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [0]:
def accuracy(y_test,y_pred):
  TP = FN = FP = TN = 0
  for j in range(len(y_test)):
      if y_test[j]== 1 and y_pred[j] == 1:
          TN = TN+1
      elif y_test[j]== 1 and y_pred[j] == -1:
          FP = FP+1
      elif y_test[j]== -1 and y_pred[j] == 1:
          FN = FN+1
      else:
          TP = TP +1
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  return (precision,recall)

In [12]:
gamma=[0.1,0.075,0.05]
nu=[0.1,0.05,0.01]
max_pre = 0
max_rec = 0
for u in gamma:
  for z in nu:
    from sklearn.svm import OneClassSVM
    classifier = OneClassSVM(kernel='rbf',nu=z,gamma=u)
    classifier.fit(X_train)
    y_pred = classifier.predict(X_test)
    (pre,rec)=accuracy(y_test,y_pred)
    if(max_rec<=rec and max_pre<=pre):
      max_rec = rec
      max_pre = pre
      print("nu= ",z," gamma: ",u)
      print(pre,rec)    

nu=  0.1  gamma:  0.1
0.7603576101177497 0.9997133027522935
nu=  0.05  gamma:  0.1
0.8678446988551518 0.9997133027522935
nu=  0.01  gamma:  0.1
0.9696885428253615 0.9997133027522935
nu=  0.01  gamma:  0.075
0.9704981909268021 0.9997133027522935
nu=  0.01  gamma:  0.05
0.9704981909268021 0.9997133027522935


In [0]:
from sklearn.svm import OneClassSVM
classifier = OneClassSVM(kernel='rbf',nu=0.05,gamma=0.075)
classifier.fit(X_train)
y_pred = classifier.predict(X_test)
print(y_pred)
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
probs = classifier.predict_proba(X_test)
probs = probs[:, -1]
auc = roc_auc_score(testy, probs)
print('AUC: %.3f' % auc)

[ 1  1  1 ... -1 -1 -1]


AttributeError: ignored

In [0]:
fpr, tpr, thresholds = roc_curve(testy, probs)